In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--headless')
options.add_argument('--disable-gpu')

In [2]:
driver = webdriver.Chrome(options=options)
driver.set_window_size(1920, 1080)

In [3]:
class LocalStorage:
    def __init__(self, driver) :
        self.driver = driver
        self.recreate_localStorage_script = '''
    const iframe = document.createElement('iframe');
    document.head.append(iframe);
    const pd = Object.getOwnPropertyDescriptor(iframe.contentWindow, 'localStorage');
    iframe.remove();
    Object.defineProperty(window, 'localStorage', pd);
    '''
        self.driver.execute_script(self.recreate_localStorage_script)

    def __len__(self):
        self.driver.execute_script(self.recreate_localStorage_script)
        return self.driver.execute_script("return window.localStorage.length;")

    def items(self):
        self.driver.execute_script(self.recreate_localStorage_script)
        return self.driver.execute_script(
            "var ls = window.localStorage, items = {}; "
            "for (var i = 0, k; i < ls.length; ++i) "
            "  items[k = ls.key(i)] = ls.getItem(k); "
            "return items; ")

    def keys(self):
        self.driver.execute_script(self.recreate_localStorage_script)
        return self.driver.execute_script(
            "var ls = window.localStorage, keys = []; "
            "for (var i = 0; i < ls.length; ++i) "
            "  keys[i] = ls.key(i); "
            "return keys; ")

    def get(self, key):
        self.driver.execute_script(self.recreate_localStorage_script)
        return self.driver.execute_script("return window.localStorage.getItem(arguments[0]);", key)

    def set(self, key, value):
        self.driver.execute_script(self.recreate_localStorage_script)
        self.driver.execute_script("window.localStorage.setItem(arguments[0], arguments[1]);", key, value)

    def has(self, key):
        return key in self.keys()

    def remove(self, key):
        self.driver.execute_script(self.recreate_localStorage_script)
        self.driver.execute_script("window.localStorage.removeItem(arguments[0]);", key)

In [4]:
home = 'https://discord.com/login?redirect_to=%2Fdevelopers%2Fapplications'
driver.get(home)
localstorage = LocalStorage(driver)

In [5]:
email = WebDriverWait(driver, 30).until(lambda x: driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[1]/div/div[2]/div/div/div/form/div[2]/div/div[1]/div[2]/div[1]/div/div[2]/input'))
driver.implicitly_wait(3)  # wait 3 more sec for everything to load
password = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[1]/div/div[2]/div/div/div/form/div[2]/div/div[1]/div[2]/div[2]/div/input')
email.send_keys('email')
password.send_keys('password')
driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[1]/div/div[2]/div/div/div/form/div[2]/div/div[1]/div[2]/button[2]').click()

In [6]:
import pickle
with open('localstorage.pkl', 'wb+') as f:
    pickle.dump(localstorage.items(), f)

In [12]:
all_applications_div = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]')
apps_str = []
apps_link = []
app_ids = []
for app_div in all_applications_div.find_elements(By.XPATH, '//*[@id="app-mount"]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div'):
    apps_str.append(app_div.find_element(By.TAG_NAME, 'abbr').text)
    apps_link.append(app_div.find_element(By.TAG_NAME, 'a').get_property('href') + '/bot')
    app_ids.append(app_div.find_element(By.TAG_NAME, 'a').get_property('href').split('/')[-1])

In [13]:
driver.get(apps_link[0])
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[2]/form/div[1]/div[1]/div[2]/div/div[2]/div/div[2]/button').click()  # reset button
driver.find_element(By.XPATH, '/html/body/div[4]/div/footer/button[2]').click()  # confirm button
token_div = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[2]/form/div[1]/div[1]/div[2]/div/div[2]/div')
new_token = token_div.text.split('\n')[2]

In [19]:
apps_link

['https://discord.com/developers/applications/1125982166371794995/bot',
 'https://discord.com/developers/applications/1125982203030020227/bot',
 'https://discord.com/developers/applications/1125982229869383821/bot',
 'https://discord.com/developers/applications/1125982259036573796/bot',
 'https://discord.com/developers/applications/1125982290644844655/bot',
 'https://discord.com/developers/applications/1125982327957372949/bot',
 'https://discord.com/developers/applications/1125982358793887836/bot',
 'https://discord.com/developers/applications/1125982385670987806/bot',
 'https://discord.com/developers/applications/1125982415463137351/bot',
 'https://discord.com/developers/applications/1125982452985380894/bot']